# 3.6 예제: 레스토랑 리뷰 감성 분류하기

이 프로젝트는 리뷰와 감성 레이블(긍정 or 부정)이 쌍을 이루는 옐프 데이터셋을 사용합니다.  
데이터를 정제하고, 훈련, 검증, 테스트 세트로 나누는 전처리 단계 및 몇가지를 추가로 설명하면서 프로젝트를 진행합니다.

앞으로 매번 사용할 3개의 보조 클래스에 대해서 간단하게 설명해보면,  
1. Vocabularay = 샘플과 타깃의 인코딩에서 설명한 정수와 토큰 매핑을 수행
2. Vectorizer = 어휘 사전을 캡슐화 하고 리뷰 텍스트 같은 문자열 데이터를 훈련과정에서 사요할 수치 벡터로 전환.
3. DataLoader = 개별 벡터 데이터 포인트를 미니 배치로 모으는 역할

# 3.6.1 옐프 리뷰 데이터셋

해당 프로젝트에서는 데이터셋의 훈련 샘플의 10%만 사용합니다. (Light version)   
이렇게 작은 데이터셋을 사용하면 훈련과 테스트가 빨라지지만 전체 데이터셋을 사용할 때 보다 낮은 정확도를 가집니다.

데이터셋을 훈련,검증,테스트용으로 나눌겁니다.  
훈련세트로 모델을 훈련하고, 검증세트로 모델이 얼마나 잘 작동하는지 평가합니다.  
검증 세트를 기반으로 모델을 선택하게 되면 물가피하게 모델이 검증세트에 더 잘 수행되도록 편향되기 때문에 모델이 점차 개선되는지 재평가 해보기 위해서 세번째 세트인 평가세트를 활용해서 이 문제를 해결합니다.

In [3]:
# import 
import collections
import numpy as np
import pandas as pd
import re

from argparse import Namespace

In [24]:
# 
args = Namespace(
    raw_train_dataset_csv = "raw_train.csv",
    raw_test_dataset_csv = "raw_test.csv",
    proportion_subset_of_train = 0.1,
    train_proportion = 0.7,
    val_proportion = 0.15,
    test_proportion = 0.15,
    output_munged_csv = "reviews_with_splits_lite.csv",
    seed = 1337
)

인자와 부속명령을 위한 명령행 옵션인, argparse를 활용해서 파이썬 내부에서 파일을 다운받고, 데이터를 나누어 줄 수 있습니다.  
train 70%, validation 15%, test 15%로 데이터를 분할할 예정입니다.

In [16]:
# 원본 데이터를 읽습니다.
train_reviews = pd.read_csv(args.raw_train_dataset_csv, header=None, names=['rating', 'review'])

다운받은 원본데이터를 파이썬에서 활용하기 위해 train_reviews 변수에 데이터를 읽어주고 컬럼명을 rating과 review 로 지정해주었다.

In [18]:
# 리뷰 클래스 비율이 동일하도록 만듭니다.
by_rating = collections.defaultdict(list)
for _, row in train_reviews.iterrows():
    by_rating[row.rating].append(row.to_dict())
    
review_subset = []

for _, item_list in sorted(by_rating.items()):
    n_total = len(item_list)
    n_subset = int(args.proportion_subset_of_train * n_total)
    review_subset.extend(item_list[:n_subset])
    
review_subset = pd.DataFrame(review_subset)

review_subset에 존재하는 데이터에서 10% 에 해당하는 데이터만 따로 저장해준다.  
그리고 defaultdic메서드를 활용해서 클래스별로 비율이 동일하도록 만들어 준 것이다.

In [19]:
review_subset.head()

,rating,review
0,1,"Unfortunately, the frustration of being Dr. Go..."
1,1,I don't know what Dr. Goldberg was like before...
2,1,I'm writing this review to give you a heads up...
3,1,Wing sauce is like water. Pretty much a lot of...
4,1,Owning a driving range inside the city limits ...


In [20]:
train_reviews.rating.value_counts()

1    280000
2    280000
Name: rating, dtype: int64

앞에서 클래스별로 비율이 동일하도록 만들어준 결과값입니다. (클래스별로 각각 28000개)

In [21]:
# 고유 클래스
set(review_subset.rating)

{1, 2}

클래스는 1과 2로 나누어지는 것을 볼 수 있다.  
1은 negative, 2는 positive 클래스이다.

In [34]:
# 코드 3-12 훈련,검증,테스트 세트 만들기

# 별점 기준으로 나누어, 훈련, 검증, 테스트를 만듭니다.
by_rating = collections.defaultdict(list)
for _, row in review_subset.iterrows():
    by_rating[row.rating].append(row.to_dict())
    
# 분할 데이터를 만듭니다.
final_list = []
np.random.seed(args.seed)

for _, item_list in sorted(by_rating.items()):
    np.random.shuffle(item_list)
    n_total = len(item_list)
    n_train = int(args.train_proportion * n_total)
    n_val = int(args.val_proportion * n_total)
    n_test = int(args.test_proportion * n_total)
    
    # 데이터 포인트에 분할 속성을 추가합니다.
    for item in item_list[:n_train]:
        item['split'] = 'train'
    
    for item in item_list[n_train:n_train+n_val]:
        item['split'] = 'val'
        
    for item in item_list[n_train+n_val:n_train+n_val+n_test]:
        item['split'] = 'test'
        
    # 최종 리스트에 추가합니다.
    final_list.extend(item_list)
    
# 분할 데이터를 데이터 프레임으로 만듭니다.
final_reviews = pd.DataFrame(final_list)

앞서 나왔던 args에서 지정한 7:1.5:1.5 비율로 각각 데이터를 train/val/test로 지정했고, 최종 리스트에 추가해주었다.

리스트형태의 데이터를 분석에 용이한 pandas 데이터 프레임으로 만들어준다.

In [35]:
final_reviews.split.value_counts()

train    39200
val       8400
test      8400
Name: split, dtype: int64

In [36]:
# 코드 3-13 최소한의 데이터 정제 작업
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"([.,!?])", r" \1", text)
    text = re.sub(r"[^a-zA-Z.,!?]+", r" ", text)
    return text

final_reviews.review = final_reviews.review.apply(preprocess_text)

최소한의 데이터 정제작업을 거친다.  
정규식을 활용하여 기호 앞뒤에 공백을 넣고, 구두점이 아닌 기호를 제거하는 정제작업을 진행해주었다.

In [37]:
final_reviews['rating'] = final_reviews.rating.apply({1: 'negative', 2: 'positive'}.get)

In [40]:
final_reviews.head()

,rating,review,split
0,negative,terrible place to work for i just heard a stor...,train
1,negative,"hours , minutes total time for an extremely s...",train
2,negative,my less than stellar review is for service . w...,train
3,negative,i m granting one star because there s no way t...,train
4,negative,the food here is mediocre at best . i went aft...,train


In [41]:
final_reviews.to_csv(args.output_munged_csv, index=False)